In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import gzip
import time
import math
import spacy
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

In [3]:
SEED=123
torch.manual_seed(SEED)
#torch.backends.cudnn.deterministic = True

In [10]:
#load the dataset
class Multi30kDataset(Dataset):
    def __init__(self):
        pass

In [12]:
def create_vocab():
    pass

In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self):
        pass

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        pass

In [14]:
class PositionwiseFeedforward(nn.Module):
    def __init__(self):
        pass

In [15]:
class EncoderLayer(nn.Module):
    def __init__(self):
        pass
    

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self):
        pass

In [16]:
class Transformer(nn.Module):
    def __init__(self):
        pass